# Uniswap V3 liquidity analysis

In this notebook we will show how to download liquidity events from Uniswap V3 to your computer as CSV files and use them to construct and analyse liquidity in each pool.

* For more background information, see [this blog post about the topic](https://tradingstrategy.ai/blog/uniswap-data-research-with-jupyter-and-python)

* You need to understand [Jupyter Notebook and Pandas basics](https://jakevdp.github.io/PythonDataScienceHandbook/)

* You need to understand [Ethereum](https://github.com/ethereumbook/ethereumbook) and [Web3.py basics](https://web3py.readthedocs.io/)

* You will need to have [Ethereum API node and its JSON-RPC URL](https://ethereumnodes.com/) in order to pull out the data from Ethereum blockchain. The notebook will interactively ask you for your API key.

* You can click *Launch binder* button on this documentation page to automatically open this notebook to be run in
  [Binder cloud notebook service](https://mybinder.org/). Note that we recommend to run the notebook
  on your local computer, as generated CSV needs are large and take long time to generate.

## Setting up the notebook

Here we import the functions we are going to need to get the data from Ethereum blockchain
to our own computer.

In [2]:
import random

import pandas as pd
import matplotlib.pyplot as plt

from eth_defi.uniswap_v3.constants import UNISWAP_V3_FACTORY_CREATED_AT_BLOCK
from eth_defi.uniswap_v3.events import fetch_events_to_csv
from eth_defi.uniswap_v3.liquidity import create_tick_delta_csv, create_tick_csv, get_pool_state_at_block

### Download the raw data

For simplicity, you can sign up for free access to an Ethereum node for example at [Infura](https://infura.io/), however we recommend you to [run your own Ethereum node](https://tradingstrategy.ai/blog/preparing-a-server-for-hosting-goethereum-full-node)

In [3]:
json_rpc_url = input("Please enter your JSON-RPC URL here")

As an example, here we download raw liquidity events from first 10000 blocks after Uniswap V3 factory is created, the events will be stored in several CSV files at `/tmp` folder

In [4]:
start_block = UNISWAP_V3_FACTORY_CREATED_AT_BLOCK
end_block = UNISWAP_V3_FACTORY_CREATED_AT_BLOCK + 10_000
fetch_events_to_csv(json_rpc_url, start_block=start_block, end_block=end_block)

TypeError: fetch_events_to_csv() missing 1 required positional argument: 'state'

### Construct intermediate data

A single raw liquidity event only tells us what happened to specific ticks in a specific pool. To be able to know how much liquidity is in each tick, we first need to construct intermediate data (we call them tick delta events) based on mint and burn events 

In [ ]:
tick_delta_csv = create_tick_delta_csv("/tmp/uniswapv3-Mint.csv", "/tmp/uniswapv3-Burn.csv")

Then we can use these delta events to aggregate into each tick liquidity 

In [ ]:
tick_csv = create_tick_csv(tick_delta_csv)

Create a dataframe we can use for further analysis

In [ ]:
tick_df = pd.read_csv(tick_csv)
tick_df

### Liquidity analysis example

Choose a random pool from the tick dataframe, for example: UNI/ETH 0.3%

In [ ]:
pool_address = "0x1d42064fc4beb5f8aaf85f4617ae8b3b5b8bd801"

Extract only the ticks relate to the pool we chose from the dataframe

In [ ]:
df = tick_df[tick_df.pool_contract_address == pool_address].sort_values(by="tick_id")
df.index = df.tick_id

df

We then can plot the liquidity gross in this pool

In [ ]:
plt.rcParams["figure.dpi"] = 200

df["liquidity_gross_delta"].astype(float).plot()

### Compare the data with The Graph data

One way to verify whether the data we construct is correct is to compare it with official [Uniswap V3 subgraph data](https://thegraph.com/hosted-service/subgraph/uniswap/uniswap-v3)  

In [ ]:
tickdelta_df = pd.read_csv(tick_delta_csv)
last_processed_block = tickdelta_df[tickdelta_df.pool_contract_address == pool_address].tail(1).block_number
last_processed_block = int(last_processed_block.values[0])

Get pool's state at the same last processed block

In [ ]:
pool_state = get_pool_state_at_block(pool_address, last_processed_block)

Then compare random ticks between our CSV data and the subgraph data

In [ ]:
ticks = pool_state["ticks"]

# get some random ticks from subgraph
for i in range(100):
    random_tick = random.choice(ticks)
    print(random_tick)

    # get the same tick from dataframe
    random_tick_df = df[df.tick_id == int(random_tick["tickIdx"])]

    # compare data
    assert int(random_tick_df.liquidity_gross_delta.values[0]) == int(random_tick["liquidityGross"])
    assert int(random_tick_df.liquidity_net_delta.values[0]) == int(random_tick["liquidityNet"])